In [81]:
%matplotlib inline
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
from scipy.linalg import eig
import tensorflow as tf
from sklearn import feature_extraction
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D,AveragePooling2D, GlobalMaxPool2D
import embed

In [77]:
def get_data(filename):
    img = nib.load(filename)
    return img.get_data(), img.affine
def scale(array):
    return (array-np.min(array))/(np.max(array) - np.min(array))
mri_data = get_data('case_001_2.nii.gz')[0].swapaxes(0,2)
labelled_data = get_data('tumor.nii.gz')[0].swapaxes(0,2)
shape = labelled_data.shape
#mri_data = mri_data.reshape((shape[0],shape[1],shape[2],1))

In [79]:
slice = mri_data[25]
x,y = np.meshgrid(np.arange(slice.shape[0]),np.arange(slice.shape[1]))
x = scale(x.reshape(x.size,1))
y = scale(y.reshape(y.size,1))
slice = scale(slice.reshape(slice.size,1)) 
X = np.column_stack((x,y,slice)).astype('float32')

In [76]:
mri_data.shape

(3041280, 1)

In [73]:
X = np.column_stack((x,y,mri_data)).astype('float32')

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [3]:
# output = 1 if slice has a tumor
y_train = np.sum(labelled_data,axis=(1,2))>0

In [82]:
# From https://github.com/satra/mapalign/blob/master/mapalign/dist.py
def compute_affinity(X, method='markov', eps=None):
    import numpy as np
    from sklearn.metrics import pairwise_distances
    D = pairwise_distances(X, metric='euclidean')
    if eps is None:
        k = int(max(2, np.round(D.shape[0] * 0.01)))
        eps = 2 * np.median(np.sort(D, axis=0)[k+1, :])**2
    if method == 'markov':
        affinity_matrix = np.exp(-(D * D) / eps)
    elif method == 'cauchy':
        affinity_matrix = 1./(D * D + eps)
    return affinity_matrix

ValueError: operands could not be broadcast together with shapes (92160,) (3,) (92160,) 

In [80]:
K = compute_affinity(X)

MemoryError: 

In [22]:
D = np.diag(np.sum(K,axis=1))

In [23]:
D_1 = np.linalg.inv(D)

In [24]:
K2= D_1.dot(K).dot(D_1)
D2 = np.diag(np.sum(K2,axis=1))

In [25]:
P = np.linalg.inv(D2).dot(K2)

In [49]:
w,vl, vr =eig(P, left=True)

In [66]:
w[::-1]

array([ -4.71903365e-85 +0.00000000e+00j,
        -6.01196773e-85 +0.00000000e+00j,
         1.19504752e-85 +0.00000000e+00j,
         2.77576705e-85 +0.00000000e+00j,
        -6.35303111e-83 +0.00000000e+00j,
        -5.06136843e-73 +0.00000000e+00j,
         5.81909412e-71 +0.00000000e+00j,
         1.02667485e-70 -2.19822840e-70j,
         1.02667485e-70 +2.19822840e-70j,
         1.33130445e-69 +0.00000000e+00j,
         2.37479233e-69 +0.00000000e+00j,
        -1.01048733e-69 -2.95562012e-69j,
        -1.01048733e-69 +2.95562012e-69j,
        -4.60480912e-69 +0.00000000e+00j,
        -1.18128496e-69 -8.35018318e-69j,
        -1.18128496e-69 +8.35018318e-69j,
         7.51152332e-69 -3.55871245e-69j,
         7.51152332e-69 +3.55871245e-69j,
        -1.61017085e-68 +0.00000000e+00j,
        -9.32626685e-69 +0.00000000e+00j,
         1.62191097e-60 +0.00000000e+00j,
         1.65261217e-54 +0.00000000e+00j,
         1.41163835e-53 -1.23214093e-53j,
         1.41163835e-53 +1.2321409

In [44]:
# get embedding
dim = 50
emb = np.zeros((dim,dim))


In [4]:
model = Sequential()
model.add(Conv2D(100,(5,5),input_shape=(shape[1],shape[2],1)))
model.add(Activation('relu'))
model.add(GlobalMaxPool2D())
#model.add(Dense(20,activation='relu'))
model.add(Dense(30,activation='relu'))
model.add(Dense(1,activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',metrics=['accuracy'])
print model.summary()
model.fit(mri_data,y_train,batch_size=2,epochs=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 316, 284, 100)     2600      
_________________________________________________________________
activation_1 (Activation)    (None, 316, 284, 100)     0         
_________________________________________________________________
global_max_pooling2d_1 (Glob (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 5,661.0
Trainable params: 5,661.0
Non-trainable params: 0.0
_________________________________________________________________
None
Epoch 1/1
33/33 [==============================] - 61s - loss: 11.5945 - acc: 0.2727    


In [ ]:
predict = model.predict(mri_data)

In [ ]:
np

In [16]:
y_train

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False, False], dtype=bool)